# Spotted star

This is a simple example of how one might go about modeling rotational light curves of spotted stars.

**NOTE: The next version of** `starry` **will likely implement actual star spots and differential rotation in a more user-friendly fashion. The** `add_gaussian` **method is in BETA, so stay tuned for updates to that. This notebook mostly showcases how if you're willing to put a little work in, you can use** `starry` **to do things beyond what it's designed to do!**

In [ ]:
import starry
import numpy as np
import matplotlib.pyplot as pl
from tqdm import tqdm

In [ ]:
%matplotlib inline

## Creating a map with a starspot

Let's create a 25th order map with a starspot at latitude 30$^\circ$ N and longitude 30$^\circ$ E:

In [ ]:
lmax = 25
star = starry.Map(lmax)
star.axis = [0, 1, 0]
star[0,0] = 1
star.add_gaussian(sigma=0.1, amp=-1, lat=30, lon=30)
star.show()

We can add additional spots to the map if we'd like:

In [ ]:
star.add_gaussian(sigma=0.1, amp=-0.3, lat=-10, lon=-20)
star.add_gaussian(sigma=0.1, amp=-0.5, lat=-70, lon=60)
star.show()

If stars didn't rotate differentially, computing this star's rotational light curve would be cake. Let's give it a rotation period of 1 day and see what it looks like:

In [ ]:
time = np.linspace(0, 3, 1000)
P = 1.
theta = 360 / P * time
flux = star.flux(theta=theta)
fig, ax = pl.subplots(1, figsize=(12, 4))
pl.plot(time, flux)
pl.xlabel('Time (days)', fontsize=18);
pl.ylabel('Flux', fontsize=18);

## Differential rotation

In reality, starspots at different latitudes should rotate at different rates. We can simulate this by generating different `starry` maps for each starspot, and adding the light curves together.

Let's generate a few spots of random sizes, amplitudes, and location on the star.

In [ ]:
nspots = 5
sigma = [0.1, 0.1, 0.2, 0.3, 0.1]
amp = [-0.4, -0.6, -0.04, -0.2, -0.4]
lat = np.array([0, 30, -20, 45, -60])
lon = np.array([30, 60, 120, -30, -45])
spot = [starry.Map(lmax) for n in range(nspots)]
for n in range(nspots):
    spot[n].axis = [0, 1, 0]
    spot[n].add_gaussian(sigma=sigma[n], amp=amp[n], lat=lat[n], lon=lon[n])

We can visualize what the star looks like at time $t = 0$ by adding the maps:

In [ ]:
x, y = np.meshgrid(np.linspace(-1, 1, 75), np.linspace(-1, 1, 75))
img = np.zeros_like(x)
for n in range(nspots):
    img += np.array([spot[n](x=x[j], y=y[j]) for j in range(75)])
pl.imshow(img, cmap='plasma', origin='lower');
pl.gca().axis('off');

That's a crazy looking star. For simplicity, let's give each of the spots a rotation period proportional to the latitude:

In [ ]:
P = 1 + np.abs(lat * np.pi / 180)
print(P)

Let's compute the light curve over the span of 10 days:

In [ ]:
# The angular position of the spots over time
time = np.linspace(0, 5, 1000)
theta = [360 / P[n] * time for n in range(nspots)]

# Plot the map
nimg = 16
res = 25
fig, ax = pl.subplots(1, nimg, figsize=(nimg, 1))
x, y = np.meshgrid(np.linspace(-1, 1, res), np.linspace(-1, 1, res))
for k, i in tqdm(enumerate(np.linspace(0, 1000, nimg, endpoint=False)), total=16):
    img = np.zeros_like(x)
    for n in range(nspots):
        img += np.array([spot[n](theta=theta[n][int(i)], x=x[j], y=y[j]) for j in range(res)])
    img -= np.nanmin(img)
    img /= np.nanmax(img)
    ax[k].imshow(img, cmap='plasma', origin='lower', vmax=1., vmin=0.);
    ax[k].axis('off');

# Plot the combined light curve
lc = 1 + np.sum([spot[n].flux(theta=theta[n]) for n in range(nspots)], axis=0)
fig, ax = pl.subplots(1, figsize=(15, 5))
ax.plot(time, lc);
ax.set_xlim(0,5)
ax.set_ylabel("Flux", fontsize=18);
ax.set_xlabel("Time", fontsize=18);

## Transits of spots

Let's now look at some spot transits. Let's start simple, with `spot[3]` (the really big one):

In [ ]:
star = spot[3]
star.show()

Now let's compute the transit light curve of a planet with $r = 0.1$ transiting along the $x$ axis at $y = 0.9$ (right through the middle of the spot):

In [ ]:
# Flux w/ spot
flux_spot = star.flux(ro=0.1, xo=np.linspace(-1.5,1.5,1000), yo=0.9)
flux_spot /= flux_spot[0]

# Flux w/ no spot
star_nospot = starry.Map(lmax)
star_nospot[0, 0] = 0.1
flux_nospot = star_nospot.flux(ro=0.1, xo=np.linspace(-1.5, 1.5, 1000), yo=0.9)
flux_nospot /= flux_nospot[0]

fig, ax = pl.subplots(1, figsize=(15, 5))
pl.plot(np.linspace(0, 1, 1000), flux_spot, label='Spotted');
pl.plot(np.linspace(0, 1, 1000), flux_nospot, label='No spot');
pl.legend(loc='lower left');
pl.ylabel("Flux", fontsize=18);
pl.xlabel("Time", fontsize=18);

Cool! 

Lastly, just for fun, we can compute the transit light curve for the multi-spotted star from before. Let's sum up the maps for all the spots and feed them into a single map, just like before:

In [ ]:
y = np.sum([spot[n].y for n in range(nspots)], axis=0)

In [ ]:
star = starry.Map(lmax)
star[:, :] = y
star[0,0] = 0.5

In [ ]:
star.show()

Looks good. Let's change the transit parameters slightly to get a prettier light curve:

In [ ]:
# Flux w/ spot
flux_spot = star.flux(ro=0.05, xo=np.linspace(-1.5,1.5,1000), yo=0.6)
flux_spot /= flux_spot[0]

# Flux w/ no spot
star_nospot = starry.Map(lmax)
star_nospot[0, 0] = 0.1
flux_nospot = star_nospot.flux(ro=0.05, xo=np.linspace(-1.5, 1.5, 1000), yo=0.6)
flux_nospot /= flux_nospot[0]

fig, ax = pl.subplots(1, figsize=(15, 5))
pl.plot(np.linspace(0, 1, 1000), flux_spot, label='Spotted');
pl.plot(np.linspace(0, 1, 1000), flux_nospot, label='No spot');
pl.legend(loc='lower left');
pl.ylabel("Flux", fontsize=18);
pl.xlabel("Time", fontsize=18);

Two *very big* spot crossings are clearly visible!